In [2]:
import numpy as np
import pandas as pd
import note_seq

from pathlib import PurePath

In [3]:
GROOVE_DIR = PurePath('../data/GrooveMIDI/groove')
groove_info = pd.read_csv(GROOVE_DIR / 'info.csv')

In [4]:
groove_info

,drummer,session,id,style,bpm,beat_type,time_signature,midi_filename,audio_filename,duration,split
0,drummer1,drummer1/eval_session,drummer1/eval_session/1,funk/groove1,138,beat,4-4,drummer1/eval_session/1_funk-groove1_138_beat_...,drummer1/eval_session/1_funk-groove1_138_beat_...,27.872308,test
1,drummer1,drummer1/eval_session,drummer1/eval_session/10,soul/groove10,102,beat,4-4,drummer1/eval_session/10_soul-groove10_102_bea...,drummer1/eval_session/10_soul-groove10_102_bea...,37.691158,test
2,drummer1,drummer1/eval_session,drummer1/eval_session/2,funk/groove2,105,beat,4-4,drummer1/eval_session/2_funk-groove2_105_beat_...,drummer1/eval_session/2_funk-groove2_105_beat_...,36.351218,test
3,drummer1,drummer1/eval_session,drummer1/eval_session/3,soul/groove3,86,beat,4-4,drummer1/eval_session/3_soul-groove3_86_beat_4...,drummer1/eval_session/3_soul-groove3_86_beat_4...,44.716543,test
4,drummer1,drummer1/eval_session,drummer1/eval_session/4,soul/groove4,80,beat,4-4,drummer1/eval_session/4_soul-groove4_80_beat_4...,drummer1/eval_session/4_soul-groove4_80_beat_4...,47.987500,test
...,...,...,...,...,...,...,...,...,...,...,...
1145,drummer2,drummer2/session2,drummer2/session2/11,rock,130,beat,4-4,drummer2/session2/11_rock_130_beat_4-4.mid,NaN,1.909613,train
1146,drummer2,drummer2/session2,drummer2/session2/12,rock,130,beat,4-4,drummer2/session2/12_rock_130_beat_4-4.mid,NaN,1.808652,train
1147,drummer2,drummer2/session2,drummer2/session2/13,rock,130,beat,4-4,drummer2/session2/13_rock_130_beat_4-4.mid,NaN,1.864421,train
1148,drummer2,drummer2/session2,drummer2/session2/14,rock,130,beat,4-4,drummer2/session2/14_rock_130_beat_4-4.mid,NaN,1.875960,train


In [5]:
hip_hops = groove_info[groove_info['style'] == 'hiphop']
hip_hops

,drummer,session,id,style,bpm,beat_type,time_signature,midi_filename,audio_filename,duration,split
573,drummer3,drummer3/session2,drummer3/session2/25,hiphop,92,beat,4-4,drummer3/session2/25_hiphop_92_beat_4-4.mid,drummer3/session2/25_hiphop_92_beat_4-4.wav,39.133157,validation
574,drummer3,drummer3/session2,drummer3/session2/26,hiphop,92,beat,4-4,drummer3/session2/26_hiphop_92_beat_4-4.mid,drummer3/session2/26_hiphop_92_beat_4-4.wav,59.974193,train
580,drummer3,drummer3/session2,drummer3/session2/31,hiphop,92,beat,4-4,drummer3/session2/31_hiphop_92_beat_4-4.mid,drummer3/session2/31_hiphop_92_beat_4-4.wav,12.975545,train
581,drummer3,drummer3/session2,drummer3/session2/32,hiphop,92,beat,4-4,drummer3/session2/32_hiphop_92_beat_4-4.mid,drummer3/session2/32_hiphop_92_beat_4-4.wav,10.376360,train
582,drummer3,drummer3/session2,drummer3/session2/33,hiphop,92,beat,4-4,drummer3/session2/33_hiphop_92_beat_4-4.mid,drummer3/session2/33_hiphop_92_beat_4-4.wav,5.161685,train
...,...,...,...,...,...,...,...,...,...,...,...
1055,drummer8,drummer8/session1,drummer8/session1/3,hiphop,90,beat,4-4,drummer8/session1/3_hiphop_90_beat_4-4.mid,drummer8/session1/3_hiphop_90_beat_4-4.wav,39.356964,train
1056,drummer8,drummer8/session1,drummer8/session1/4,hiphop,90,beat,4-4,drummer8/session1/4_hiphop_90_beat_4-4.mid,drummer8/session1/4_hiphop_90_beat_4-4.wav,114.480613,train
1057,drummer8,drummer8/session1,drummer8/session1/5,hiphop,90,beat,4-4,drummer8/session1/5_hiphop_90_beat_4-4.mid,drummer8/session1/5_hiphop_90_beat_4-4.wav,127.755619,train
1063,drummer8,drummer8/session2,drummer8/session2/10,hiphop,86,beat,4-4,drummer8/session2/10_hiphop_86_beat_4-4.mid,NaN,147.478109,train


In [6]:
np.random.seed(42)
h_idx = np.random.choice(hip_hops.index)
sample_track = note_seq.midi_file_to_note_sequence(GROOVE_DIR / hip_hops['midi_filename'][h_idx])

In [7]:
note_seq.plot_sequence(sample_track)

Loading BokehJS ...

In [8]:
note_seq.play_sequence(sample_track, synth=note_seq.synthesize)

/afs/inf.ed.ac.uk/user/s19/s1984690/Research/vxs-vpt/pythonenv/lib64/python3.6/site-packages/pretty_midi/pretty_midi.py:918: RuntimeWarning: invalid value encountered in true_divide
  synthesized /= np.abs(synthesized).max()
